<a href="https://colab.research.google.com/github/syeong1218/keras-fig/blob/master/keras_WGAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5장 개선된 GAN 모델

- WGAN의 이론적 방정식
- LSGAN 이론
- ACGAN 이론
- 케라스를 사용해 개선된 GAN 모델을 구현하는 방법

### GAN
판별기는 진짜 데이터에서 가짜 데이터를 정확하게 분류해내려고 하고, 생성기는 판별기를 속이려한다. 판별기와 생성기가 서로 상반된 목표를 가지고 있어 훈련할 때 쉽게 불안정해진다.

GAN의 목표는 생성기 데이터 분포를 진짜 데이터 분포로 만드는 것이다.



###  WGAN

판별기와 생성기를 교대로 훈련시킨다. 생성기를 1회 훈련시키기 전에 판별기를 $n_{critic}$회 훈련시킨다. 이는 생성기와 판별기를 동일한 횟수로 훈련시키는 GAN과 다른 점이다.

![대체 텍스트](https://github.com/syeong1218/keras-fig/blob/master/5.1.3.JPG?raw=true)
> **알고리즘**<br>
  매개변수 : $\alpha=0.00005, c=0.01, m=64, n_{critic}=5$ <br>
  조건 : $\alpha$- 학습속도, c-클리핑 매개변수, m-배치크기, $n_{critic}$-생성기가 1회 반복하는 동안 판별기의 반복 횟수<br>
  조건 : $w_0$-초기 판별기 매개변수, $\theta_0$-초기 생성기 매개변수<br>
  1. while $\theta$가 수렴하지 않는 동안 do
  2. for t=1,...,$n_{critic}$ do
  3. 실제 데이터에서 배치 ${(x^{(i)})}_{i=1}^m \sim p_{data}$를 샘플링
  4. 균등 노이즈 분포에서 배치 ${(z^{(i)})}_{i=1}^m \sim p(z)$를 샘플링
  5. 판별기 경사를 계산,$g_w\leftarrow \triangledown _w\left [ -\frac{1}{m}\sum_{i=1}^{m}D_w(x^{(i)})+\frac{1}{m}\sum_{i=1}^{m}D_w(g_\theta(z^{(i)})) \right ]$
  6. 판별기 매개변수 업데이트,$w\leftarrow w-\alpha \times RMSProp(w,g_w)$
  7. 판별기 가중치 제한, $w\leftarrow clip(w,-c,c)$
  8. end for
  9. 균등 노이즈 분포에서 배치 ${(z^{(i)})}_{i=1}^m \sim p(z)$를 샘플링
  10. 생성기 경사 계산, $g_{\theta}\leftarrow \triangledown _{\theta}\frac{1}{m}\sum_{i=1}^{m}D_w(g_\theta(z^{(i)}))$
  11. 생성기 매개변수 업데이트, $\theta\leftarrow \theta-\alpha \times RMSProp(w,g_{\theta})$
  12. end while
.

판별기를 훈련시킨다는 것은 판별기의 매개변수를 학습한다는 뜻이다.
>>**판별기 훈련**
<br>3. 실제 데이터에서 배치를 샘플링
 <br>4. 가짜 데이터에서 배치를 샘플링
 <br>5. 샘플링된 데이터를 판별기 네트워크에 공급한 다음, 판별기 매개변수의 경사를 계산
 <br>6. 판별기 매개변수는 RMSProp를 사용해 최적화
 <br>7. EMD 최적화에서 판별기 매개변수를 특정 범위 내로 제한해서 립시츠의 제약 조건을 적용
 <br><br>실제 데이터를 사용해 훈련할 때 손실 함수를 최소화하기 위해 $y_{prediction}=D_{w}(x)$를 증가시킨다. 가짜 데이터를 사용해 훈련시키는 경우, $y_{prediction}=D_{w}(G(x))$를 감소시켜 손실 함수를 최소화한다.

>>**생성기 훈련**
 <br>9. 가짜 데이터 배치를 샘플링
 <br>10. 생성기 경사 계산
 <br>11. RMSProp를 사용해 최적화
 <br><br> $y_{prediction}=D_{w}(G(x))$를 증가시켜 훈련하는 동안 가짜 데이터에 진짜 레이블이 달려 있을 때 손실함수를 최소화한다. 
 
생성기 훈련이 끝나면 판별기 매개변수를 고정했던 것을 풀고 다시 판별기 훈련을 $n_{critic}$회 시작한다. 

생성기는 데이터 위조에만 관여하기 때문에 판별기를 훈련하는 동안 생성기 매개변수를 고정할 필요가 없다.

판별기 경사를 계산할 때 실제 데이터의 레이블은 1.0이지만 가짜 데이터의 레이블은 -1.0이 된다.


- RMSProp 사용 이유

  critic을 학습 할 때 Adam과 같은 mometum 베이스 optimizer를 사용하면 학습이 불안정하다 

  그 이유는 loss값이 튀고 샘플이 좋지 않은 경우(일반적으로 학습 초반) Adam이 가고자 하는 방향, 즉 이전에 기억했던 방향(Adam step)과 gradient의 방향 간의 cosine값이 음수가 된다. 일반적으로 nonstationary 문제(극한값이 존재하지 않음)에 대해서는 momentum계열보다 RMSProp이 성능이 더 좋다고 한다.

In [1]:
try:

    %tensorflow_version 1.x  # %tensorflow_version only exists in Colab

except Exception:

    pass

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  # %tensorflow_version only exists in Colab`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
from google.colab import files
src=list(files.upload().values())[0]

Saving gan.py to gan.py


In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Input
from keras.optimizers import RMSprop
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
from keras.models import load_model

import numpy as np
import argparse

import sys
import gan

def train(models, x_train, params):
    """판별기와 적대적 네트워크 훈련
    판별기와 적대적 네트워크를 배치 단위로 교대로 훈련
    먼저 판별기가 제대로 레이블이 붙은 진짜와 가짜 이미지를 사용해 n_critic회 훈련됨
    판별기 가중치는 립시츠 제약 조건에 따라 범위가 제한됨
    다음으로 생성기가 진짜인 척하는 가짜 이미지를 사용해(적대적 네트워크를 통해) 훈련됨
    save_interval마다 샘플 이미지를 생성

    #인수
     models(list) : Generator, discriminator, Adversarial 모델
     x_train(tensor) : 이미지 훈련
     params(list) : 네트워크 매개변수
    """
    # GAN 모델
    generator, discriminator, adversarial = models
    # 네트워크 매개변수
    (batch_size, latent_size, n_critic, 
            clip_value, train_steps, model_name) = params
    # 생성기 이미지는 500 단계마다 저장됨
    save_interval = 500
    # 훈련하는 동안 생성기 출력이 어떻게 진화하는지 보기 위한 노이즈 벡터
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # 훈련 데이터 세트의 요소 개수
    train_size = x_train.shape[0]
    # 실제 데이터의 레이블
    real_labels = np.ones((batch_size, 1))
    for i in range(train_steps):
        # 판별기를 n_critic회 훈련시킴
        loss = 0
        acc = 0
        for _ in range(n_critic):
            # 1 배치에 대해 판별기를 훈련
            # 실제 이미지(label=1.0)와 가짜 이미지(label=-1.0)으로 구성된 1 배치
            # 데이터세트에서 실제 이미지를 임의로 선정
            rand_indexes = np.random.randint(0, train_size, size=batch_size)
            real_images = x_train[rand_indexes]
            # 생성기를 사용해 노이즈에서 가짜 이미지 생성
            # 균등 분포를 사용해 노이즈 생성
            noise = np.random.uniform(-1.0,
                                      1.0,
                                      size=[batch_size, latent_size])
            fake_images = generator.predict(noise)

            # 판별기 네트워크 훈련
            # 진짜 데이터 레이블l=1, 가짜 데이터 레이블=-1
            # 진짜와 가짜 이미지를 결합해 하나의 배치를 만드는 대신
            # 처음에는 진짜 데이터로 구성된 하나의 배치로 훈련한 다음
            # 가짜 이미지로 구성된 하나의 배치로 훈련
            # 이렇게 바꿈으로써
            # 진짜와 가짜 데이터 레이블의 부호가 반대고(+1과 -1)
            # 점위 제한(클리핑)으로 인해 가중치의 크기가 작아서
            # 경사가 소실되는 것을 방지
            real_loss, real_acc = discriminator.train_on_batch(real_images,
                                                               real_labels)
            fake_loss, fake_acc = discriminator.train_on_batch(fake_images,
                                                               -real_labels)
            # 평균 손실과 정확도를 누적
            loss += 0.5 * (real_loss + fake_loss)
            acc += 0.5 * (real_acc + fake_acc)

            # 립시츠 제약 사향을 만족하기 위해 판별치 가중치 범위 제한
            for layer in discriminator.layers:
                weights = layer.get_weights()
                weights = [np.clip(weight,
                                   -clip_value,
                                   clip_value) for weight in weights]
                layer.set_weights(weights)

        # n_critic회 반복 훈련하는 동안 평균 손실과 정확도
        loss /= n_critic
        acc /= n_critic
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

        # 1 배치 동안 적대적 네트워크 훈련
        # label=1.0인 가짜 이미지의 1 배치
        # 적대적 네트워크의 판별기 가중치가 고정되어 있으므로
        # 생성기만 훈련됨
        # 균등 분포를 사용해 노이즈 생성
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # 적대적 네트워크 훈련
        # 판별기 훈련과 달리 변수에 가짜 이미지를 저장하지 않음
        # 가짜 이미지는 분류를 위해 적대적 네트워크의 판별기 입력으로 전달됨
        # 가짜 이미지는 진짜 레이블을 가지고 있음
        # 손실과 정확도를 기록
        loss, acc = adversarial.train_on_batch(noise, real_labels)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            if (i + 1) == train_steps:
                show = True
            else:
                show = False

            # 주기적으로 생성기 이미지를 그림
            gan.plot_images(generator,
                            noise_input=noise_input,
                            show=show,
                            step=(i + 1),
                            model_name=model_name)

    # 생성기 훈련이 끝나면 모델을 저장
    # 훈련기 생성기는 향후 MNIST 숫자 생성을 위해 재로딩될 수 있음
    generator.save(model_name + ".h5")


def wasserstein_loss(y_label, y_pred):
    return -K.mean(y_label * y_pred)


def build_and_train_models():
    # MNIST 데이터세트 로딩
    (x_train, _), (_, _) = mnist.load_data()

    # CNN 데이터를 (28, 28, 1)로 형상을 변경하고 정규화함
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32') / 255

    model_name = "wgan_mnist"
    # 네트워크 매개변수
    # 잠재 혹은 z벡터 차원은 100
    latent_size = 100
    # WGAN 논문[2]에서 참조한 매개변수
    n_critic = 5
    clip_value = 0.01
    batch_size = 64
    lr = 5e-5
    train_steps = 400
    input_shape = (image_size, image_size, 1)

    # 판별기 모델 구성
    inputs = Input(shape=input_shape, name='discriminator_input')
    # WGAN은 논문[2]의 선형 활성화를 사용
    discriminator = gan.discriminator(inputs, activation='linear')
    optimizer = RMSprop(lr=lr)
    # WGAN 판별기는 베셔슈타인 손실을 사용
    discriminator.compile(loss=wasserstein_loss,
                          optimizer=optimizer,
                          metrics=['accuracy'])
    discriminator.summary()

    # 생성기 모델 구성
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='z_input')
    generator = gan.generator(inputs, image_size)
    generator.summary()

    # 적대적 모델 생성 = 생성기 + 판별기
    # 적대적 네트워크를 훈련하는 동안 판별기의 가중치는 고정
    discriminator.trainable = False
    adversarial = Model(inputs,
                        discriminator(generator(inputs)),
                        name=model_name)
    adversarial.compile(loss=wasserstein_loss,
                        optimizer=optimizer,
                        metrics=['accuracy'])
    adversarial.summary()

    # 판별기와 적대적 네트워크를 훈련
    models = (generator, discriminator, adversarial)
    params = (batch_size,
              latent_size,
              n_critic,
              clip_value,
              train_steps,
              model_name)
    train(models, x_train, params)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    help_ = "Load generator h5 model with trained weights"
    parser.add_argument("-g", "--generator", help=help_)
    args = parser.parse_args(args=[])
    if args.generator:
        generator = load_model(args.generator)
        gan.test_generator(generator)
    else:
        build_and_train_models()

linear
Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa (None, 28, 28, 1)         0         
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 128)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0: [discriminator loss: 0.011445, acc: 0.000000] [adversarial loss: -0.000029, acc: 0.000000]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1: [discriminator loss: -0.000068, acc: 0.000000] [adversarial loss: 0.000197, acc: 0.000000]
2: [discriminator loss: -0.000214, acc: 0.000000] [adversarial loss: 0.000566, acc: 0.000000]
3: [discriminator loss: -0.000507, acc: 0.000000] [adversarial loss: 0.001397, acc: 0.000000]
4: [discriminator loss: -0.001106, acc: 0.000000] [adversarial loss: 0.003198, acc: 0.000000]
5: [discriminator loss: -0.002618, acc: 0.000000] [adversarial loss: 0.007528, acc: 0.000000]
6: [discriminator loss: -0.005701, acc: 0.000000] [adversarial loss: 0.015940, acc: 0.000000]
7: [discriminator loss: -0.011364, acc: 0.000000] [adversarial loss: 0.029976, acc: 0.000000]
8: [discriminator loss: -0.021067, acc: 0.000000] [adversarial loss: 0.051825, acc: 0.000000]
9: [discriminator loss: -0.034554, acc: 0.000000] [adversarial loss: 0.080696, acc: 0.000000]
10: [discriminator loss: -0.050379, acc: 0.000000] [adversarial loss: 0.113871, acc: 0.000000]
11: [discriminator loss: -0.073137, acc: 0.000000] [adversa